# infos

In [ ]:
# Version 1.0 /Ce notebook n'est pas finalisé, il sert de test

# Recherche du meilleur modèle avec optuna

### Changelogs :


v1.0 :
- Construction Notebook
- Necessite debugage

# Configuration Notebook

### Environnement colab uniquement

In [1]:
# Executer uniquement cette cellule : Si environnement google Colab
print('Montage du lecteur Gdrive')
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/SAS/Jul23_bds_champignons/notebooks')

Montage du lecteur Gdrive
Mounted at /content/drive


### Config

In [8]:
# Dimensions des images à traiter et des batchs
img_dim = (256,256)
img_shape = (256,256,3)
batch_size = 64
model_name = 'V0_Optuna_tuned'
#data_file = '/content/drive/MyDrive/SAS/Jul23_bds_champignons/data/model_data_10_filtered.csv'
data_path = '../data/model_data_10_filtered.csv'
data_size = '2' # '1' pour données complètes
# chemin_images = '/images/images'
chemin_images = '../../images'
pourcentage_echantillon = 0.05
undersampling = True
work_env = '1' # '2' sur G Colab

In [2]:
if work_env == '2':

  # Importer les images en format .zip
  #from google.colab import files
  #files.upload()

  # Dezipper le fichier images dans Colab
  !unzip '/content/drive/MyDrive/SAS/images.zip' -d '/images'

else:
    print("L'environnement choisi n'est pas Google Colab, aucune modification apportée dans les chemins fichiers.")

L'environnement choisi n'est pas Google Colab, aucune modification apportée dans les chemins fichiers.


# Librairies générales du notebook

In [3]:
# Librairies générales servant dans le notebook
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna

# Jeu de données

In [9]:
from fonctions_data import import_df
df, df_ech = import_df(chemin_images, data_path, pourcentage_echantillon)

# Le DataFrame qui sera load dans la pipeline tensorflow dépend du choix inital de taille des données :
if data_size == "1":
    training_data = df

elif data_size == "2":
    training_data = df_ech


# undersampling des classes : (si défini)
if undersampling == True:
  from fonctions_data import undersampling_df
  training_data = undersampling_df(training_data, col = 'label')

else:
  print("Pas d'undersampling programmé, les données sont déséquilibrées.")



# Controle de la présence des fichiers images
from fonctions_data import controle_presence_fichiers
controle_presence_fichiers(training_data, chemin_images)

# On supprime ensuite la colonne image_lien qui ne sert qu'à controler la présence des fichiers.
# On supprime également la colonne 'saison' qui ne servira pas pour l'entrainement.
training_data.drop(['image_lien', 'saison'], axis=1, inplace=True)

print("\n\n Nombre d'images par classes :")
print(training_data.groupby('label').count())

training_data.head()

Nombre d'images chargées pour df: 49861
Nb especes dans df: 10
Nombre d'images chargées pour df_ech: 2493
Nb especes dans df_ech: 10

Tous les fichiers sont présents.


 Nombre d'images par classes :
             image_url
label                 
Agaricus           167
Amanita            167
Cortinarius        167
Entoloma           167
Gymnopilus         167
Inocybe            167
Mycena             167
Pluteus            167
Psathyrella        167
Russula            167


,label,image_url
0,Agaricus,../../images/32247.jpg
1,Agaricus,../../images/67307.jpg
2,Agaricus,../../images/502889.jpg
3,Agaricus,../../images/355520.jpg
4,Agaricus,../../images/574033.jpg


# Pipeline images

In [11]:
# Construction des datasets
from fonctions_dataset import gen_dataset, load_image
dataset_train = gen_dataset(set = 'train', training_data = training_data, img_dim = img_dim, augment = True, normalize = False, batch_size = batch_size)
dataset_val = gen_dataset(set = 'validation', training_data = training_data, img_dim = img_dim, augment = False, normalize = False, batch_size = batch_size)
dataset_test = gen_dataset(set = 'test', training_data = training_data, img_dim = img_dim, augment = False, normalize = False, batch_size = batch_size)

target encodée: [0 0 0 ... 9 9 9]
target encodée: [0 0 0 ... 9 9 9]
target encodée: [0 0 0 ... 9 9 9]


# Construction modèle

In [14]:
from optuna_fonctions import *

study = optuna.create_study(direction='maximize')

[I 2023-10-09 16:04:09,259] A new study created in memory with name: no-name-c7667619-82c7-4496-a553-fb74f01c2908


# Entrainement du modèle

In [15]:
study.optimize(objective(trial, epochs = 3), n_trials=1, n_jobs=-1)

NameError: name 'trial' is not defined

# Optuna Resultats

In [ ]:
best_params = study.best_params
best_value = study.best_value
print(best_params)

In [ ]:
optuna_viz.plot_param_importances(study)

In [ ]:
# Compiler le modèle
best_model = create_model(study.best_trial)
best_model.summary()

# Evaluation

In [ ]:
import matplotlib.pyplot as plt

# Courbes d'évolution de l'accuracy et de la val_accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Courbes d'évolution de la loss et de la val_loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show();

In [ ]:
test_results = model.evaluate(dataset_test)
print("Resultats de la prediction:", test_results)

y_pred = model.predict(dataset_test)

In [ ]:
# Rapport de classification
y_pred_classes = np.argmax(y_pred, axis=1)

from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred_classes)
print(report)


In [ ]:
# Matrice de confusion
import seaborn as sns
from sklearn.metrics import confusion_matrix

confusion_mtx = confusion_matrix(y_test, y_pred_classes)
# Créer une figure pour afficher la matrice de confusion
plt.figure(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.title('Matrice de confusion')
plt.show()

# Sauvegarde du modèle

In [ ]:
if work_env == '2':
    model_name = '/content/drive/MyDrive/SAS/model/' + model_name
    history_path = '/content/drive/MyDrive/SAS/history/history.pkl'


else:
    model_name = '../model/' + model_name
    history_path = '../history/history.pkl'


model.save(model_name, save_format='h5')


import pickle
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)